In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
Datadirectory = "train/" ##  training dataset

In [3]:
Classes = ["0","1","2","3","4","5","6"] ##list of classes angry, disgust, fear, happy, neutral, sad, surprise

<h1>Reading in all the images into our array.</h1>

In [4]:
training_Data = [] ##data array to train
img_size = 224 #image size used for our pre trained modal

def create_training_data():
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        Class = category
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (img_size,img_size))
                training_Data.append([new_array,Class])
            except Exception as e:
                pass

In [5]:
create_training_data()

In [6]:
import random

random.shuffle(training_Data)

x = []  ## Data
y = []  ## Label

##Normalize the data by dividint it with the highest number which is 255 meaning black

##This step takes alot of ram which my computer cannot handle therefore the following code
##lowers the number of data set to be compatible for ram
count = 0
for features, label in training_Data:
    if (label == 'Fear') or (count >= 20000): #Removing 17,500 to compensate for the lack of RAM
        x.append(features)
        y.append(label)
    else:
        count = count + 1
        continue
        
x = np.array(x).reshape(-1 , img_size, img_size, 3) ##converting to 4 dimension as the transfer learning modal we use called mobilenet which has a deep learning architechture

In [7]:
x = (x/255.0)*1.0

<h1>Training the deep learining model</h1>

In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = tf.keras.applications.MobileNetV2() ##Pre-trained Model

In [9]:
base_input = model.layers[1].input
base_output = model.layers[-2].output

In [10]:
final_output = layers.Dense(128)(base_output)  ## adding new layer after the output of global pooling layer
final_output = layers.Activation('relu')(final_output) ##activatiuon function 
final_output = layers.Dense(64)(final_output) 
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7, activation='softmax')(final_output) #decreasing the number of classes(Categories) to 7

new_model = keras.Model(inputs = base_input, outputs = final_output)

In [11]:
new_model.compile(loss="sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
x = np.array(x)
y = np.array(y)

In [12]:
y = y.astype(int)

In [14]:
new_model.fit(x,y, epochs = 25)

Epoch 1/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 392s 1s/step - accuracy: 0.9474 - loss: 0.1557
Epoch 2/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.9443 - loss: 0.1644
Epoch 3/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 386s 1s/step - accuracy: 0.9360 - loss: 0.1820
Epoch 4/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.9591 - loss: 0.1210
Epoch 5/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.9560 - loss: 0.1319
Epoch 6/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 384s 1s/step - accuracy: 0.9180 - loss: 0.2301
Epoch 7/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.9330 - loss: 0.1869
Epoch 8/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 384s 1s/step - accuracy: 0.9492 - loss: 0.1671
Epoch 9/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.9663 - loss: 0.0985
Epoch 10/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 385s 1s/step - accuracy: 0.9444 - loss: 0.1704
Epoch 11/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 388s 1s/step - accuracy: 0.9585 - loss: 0.1193
Epoch 12/25
273/273 ━━━━━━━━━━━━━━━━━━━━ 

In [15]:
new_model.save('Final_model.h5')

In [16]:
model.save('my_model.keras')

In [5]:
import cv2
import tensorflow as tf
import numpy as np
import pandas as pd
new_model = tf.keras.models.load_model('Final_model.h5')

# Load pre-trained face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start capturing video from the default camera
cap = cv2.VideoCapture(0)


while True:
    # Read a frame from the camera
    ret, frame = cap.read()
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    # Loop through each face and crop it
    for (x, y, w, h) in faces:
        # Crop the face region from the frame
        face_crop = frame[y:y+h, x:x+w]
        
        # Display the cropped face
        cv2.imshow('Face Crop', face_crop)
        
    # Display the original frame
    cv2.imshow('Original', frame)
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    #roi_color = frame[y:y+h,x:x+w]
    #face_roi = crop_img[ey: ey+eh, ex: ex+ew]
    final_image = cv2.resize(face_crop, (224, 224))
    final_image = np.expand_dims(final_image, axis = 0)
    final_image = final_image/255.0


    Predictions = new_model.predict(final_image)

    font_scale = 1.5
    font = cv2.FONT_HERSHEY_PLAIN
    #Classes = ["0","1","2","3","4","5","6"] ##list of classes angry, disgust, fear, happy, neutral, sad, surprise
    if (np.argmax(Predictions) == 0):
        status = "Angry"
    elif (np.argmax(Predictions) == 1):
        status = "Disgust"
    elif (np.argmax(Predictions) == 2):
        status = "Fear"
    elif (np.argmax(Predictions) == 3):
        status = "Happy"
    elif (np.argmax(Predictions) == 4):
        status = "Neutral"
    elif (np.argmax(Predictions) == 5):
        status = "Sad"
    else:
        status = "Surprise"
        
    x1, y1,w1,h1 = 0,0,175,75
    cv2.rectangle(frame, (x1,x1), (x1 + w1, y1+h1), (0,0,0), -1)
    cv2.putText(frame, status, (x1 + int(w1/10), y1 + int(h1/2)),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
    cv2.putText(frame, status, (100,150),font, 3, (0,0,255),2,cv2.LINE_4)
    cv2.rectangle(frame, (x,y), (x+w, y+h), (0,0,255))
    
    cv2.imshow('Face Emotion Recognition', frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

# Release the VideoCapture object
cap.release()

# Destroy all windows
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━